In [2]:
import numpy as np
from termcolor import colored
import collections

from astropy.table import Table, join, vstack, Column
from astroquery.alma import Alma
from astropy import coordinates
from astropy import units as u
from astropy.time import Time
from astroquery.vizier import Vizier
from astroquery.ipac.irsa import Irsa
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad

import warnings
warnings.filterwarnings("ignore", category=Warning, module='astropy.units.format.vounit')
warnings.filterwarnings("ignore", category=Warning, module='astroquery.ipac.irsa.core')
warnings.filterwarnings("ignore", category=Warning, module='astropy.io.fits.column')
warnings.filterwarnings("ignore", category=Warning, module='astropy.utils.metadata')

# Import Functions

In [3]:
def get_2mass_from_coords(name, ra, de, coord_form='hmsdms', arcsec_rad = 2.0):
    
    _2mass = []
    name, ra, de =  np.atleast_1d(np.array(name)), np.atleast_1d(np.array(ra)), np.atleast_1d(np.array(de))
    coord = set_coord(coord_form, ra, de)
    for i, val in enumerate(name):
        
        try:
            val = val.decode('utf-8')
        except AttributeError:
            val = val
        
        r = Irsa.query_region(coord[i], catalog="fp_psc", spatial="Cone", radius=arcsec_rad*u.arcsec)
        if len(r) == 1:
            _2mass.append('J' + r['designation'][0])
            if r['dist'][0] > 2.0:
                print(colored(f"{val}: large separation match {r['dist'][0]} arcsec", 'yellow'))
            # print(f"{val}: found")
            
        else:
            print(colored(f"{val}: could not find 2MASS match within {arcsec_rad} arcsec", 'red'))
            _2mass.append('--')
   
    return _2mass

In [4]:
def get_coords_from_2mass(id_2mass, coord_form='hmsdms'):

    ra, de, = [], []
    id_2mass = np.atleast_1d(np.array(id_2mass))
    for i, val in enumerate(id_2mass):
        
        try:
            val = val.decode('utf-8')
        except AttributeError:
            val = val
            
        v = Vizier(columns=["*", "+_r"], catalog="II/246")
        r = v.query_region(val, radius="2s")[0]
        
        if len(r) == 1:
            # print(r['_r'])
            c = SkyCoord(r['RAJ2000'][0], r['DEJ2000'][0], frame='icrs', unit='deg')
            
            if coord_form == 'hmsdms':
                tmp = c.to_string('hmsdms', sep=':', precision=3)
                ra.append(tmp.split(' ')[0])
                de.append(tmp.split(' ')[1])
            elif coord_form == 'deg':
                ra.append(c.ra.deg)
                de.append(c.dec.deg)
            else:
                print(colored("coordinate type not supported, returning degrees"))
                ra.append(c.ra.deg)
                de.append(c.dec.deg)
            
        else:
            ra.append('')
            de.append('')
            print(colored(f"{val}: could not find coordinates", 'red'))
            
    return ra, de

In [5]:
def get_edr3(name, ra, de, pmra, pmde, coord_form='deg', arcsec_rad=2.0):

    ### NEEDED FOR REST OF FUNCTION TO WORK IF ONLY ONE TARGET ENTERED
    name = np.atleast_1d(np.array(name))
    ra, de = np.atleast_1d(np.array(ra)), np.atleast_1d(np.array(de))
    pmra, pmde = np.atleast_1d(np.array(pmra)), np.atleast_1d(np.array(pmde))

    ### SET COORDINATES
    coord = set_coord(coord_form, ra, de)
        
    ### LOOP THROUGH TARGETS
    source, plx, eplx, pmRA, pmDE, RUWE = [], [], [], [], [], []
    arrs = [source, plx, eplx, pmRA, pmDE, RUWE]
    cols = ['Source', 'Plx', 'e_Plx', 'pmRA', 'pmDE', 'RUWE']
    for i, val in enumerate(name):

        if (type(val) != 'str') & (type(val) != np.str_):
            val = val.decode('utf-8')
            
        ### CORRECT FOR PROPER MOTION
        pmra_deg, pmde_deg = pmra[i] * (1 / 3600000.) / np.cos(np.radians(coord[i].dec.deg)), pmde[i] * (1 / 3600000.)
        mjd_diff = (57023.5 - 51544.5) / 365.
        ra_deci, de_deci = coord[i].ra.deg + pmra_deg * mjd_diff, coord[i].dec.deg + pmde_deg * mjd_diff

        ### CONDUCT QUERY AND SAVE VALUES
        VSearch = Vizier(columns=["+_r", "*"])
        r = VSearch.query_region(SkyCoord(ra_deci, de_deci, unit=(u.deg, u.deg)), 
                                 radius=arcsec_rad*u.arcsec, catalog="I/350/gaiaedr3")
        if len(r) == 0:
            print(colored(f"{val}: could not find EDR3 match within {arcsec_rad} arcsec", 'red'))
            for n, nval in enumerate(cols):
                arrs[n].append(-99)
        else:
            r = r['I/350/gaiaedr3']
            for n, nval in enumerate(cols):
                r[nval].fill_value = -99.0
                arrs[n].append(r[nval].filled()[0])
            
    return arrs

In [6]:
def get_pm_from_simbad(name, ra, de, coord_form='hmsdms', arcsec_rad=2.0):
    
    #### POSSIBLE ADD-ONS FOR QUERY
    ### Simbad.list_votable_fields()
    ### Simbad.get_field_description(FIELDNAME)

    ### NEEDED FOR REST OF FUNCTION TO WORK IF ONLY ONE TARGET ENTERED
    name, ra, de = np.atleast_1d(np.array(name)), np.atleast_1d(np.array(ra)), np.atleast_1d(np.array(de))

    ### SET COORDINATES
    coord = set_coord(coord_form, ra, de)
        
    ### QUERY SIMBAD FOR PROPER MOTIONS
    pmra, pmde = [], []
    for i, val in enumerate(name):

        ### FIX NAME TO STRING
        if (type(val) != 'str') & (type(val) != np.str_):
            val = val.decode('utf-8')

        ### ADD PM TO SIMNBAD QUERY
        customSimbad = Simbad()
        customSimbad.add_votable_fields('ids')
        customSimbad.add_votable_fields('pm')
        customSimbad.add_votable_fields('distance_result')

        ### CONDUCT QUERY AND SAVE VALUES
        r = customSimbad.query_region(SkyCoord(coord[i].ra.deg, coord[i].dec.deg, unit=(u.deg, u.deg), frame='icrs'), radius=arcsec_rad * u.arcsec)
        if r is None:
            print(colored(f"{val}: could not find SIMBAD match within {arcsec_rad} arcsec\n", 'red'))
            pmra.append(-99.0)
            pmde.append(-99.0)
        else:
            if len(r) > 1:
                r['PMRA'].fill_value, r['PMDEC'].fill_value, r['IDS'].fill_value = -99.0, -99.0, ''
                msk, ft, fc = [val.replace(' ','') in x.replace(' ','') for x in r['IDS'].filled().data], 'keeping closest match', 'yellow'
                if True in msk:
                    r, ft, fc = r[msk], 'keeping name match', 'blue'
                print(colored(f"\n{val}: found multiple matches within {arcsec_rad} arcsec, {ft}", fc))
                print(r['MAIN_ID', 'DISTANCE_RESULT', 'PMRA', 'PMDEC'])
            r = r[np.where(r['DISTANCE_RESULT'] == np.min(r['DISTANCE_RESULT']) )]
            r['PMRA'].fill_value, r['PMDEC'].fill_value = -99.0, -99.0
            pmra.append(r['PMRA'].filled()[0])
            pmde.append(r['PMDEC'].filled()[0])
            
    return pmra, pmde

In [7]:
def set_coord(coord_form, ra, de):
    
    if coord_form == 'hmsdms':
        coord = SkyCoord(ra, de, frame='icrs', unit=(u.hourangle, u.deg))
        
    elif coord_form == 'deg':
        coord = SkyCoord(ra, de, frame='icrs', unit='deg')
        
    else:
        print(colored("coordinate type not supported, this will break now", 'yellow'))
        coord, rad == '', ''

    return coord

# Ophiuchus

In [27]:
### JPW'S TABLE SHOULD HAVE ENTIRE SAMPLE A + B (SEE HIS EMAIL IN JAN 2021)
t = Table.read('./Data/2019_Williams_Oph_Table1.fit') 

### ADD SURVEY INFO
t['Survey_B6'] = 'C-W+2019'
t['Freq_B6'] = 225.4
t['Wave_B6'] = 1.33

### TRANSLATE RA & DEC FORMATS
coords = SkyCoord(t['_RA'], t['_DE'], frame='icrs', unit='deg')
t['RA_J2000'] = [x.split(' ')[0] for x in coords.to_string('hmsdms', sep=':', precision=3)]
t['DE_J2000'] = [x.split(' ')[1] for x in coords.to_string('hmsdms', sep=':', precision=3)]

### SHOULD BE 279 FOR TOTAL SAMPLE (265 UNIQUE WITH 12 BINARIES AND 1 TRIPLE)
len(t), len(np.unique(t['SSTc2d']))

(279, 265)

In [28]:
### RES AND BEAM DEPENDED ON SAMPLE A OR B
tA = Table.read('./Data/2019_Cieza_Oph_Table1.fit')
t['Sample'], t['Beam_B6'], t['RMS_B6'] = 'B', '0.98x0.74', -99.0
for i, val in enumerate(t['SSTc2d']):
    if val in tA['SSTc2d']:
        t['Sample'][i], t['Beam_B6'][i], t['RMS_B6'][i] = 'A', '0.28x0.19', 0.15 

In [29]:
### GET PROPER MOTIONS FROM SIMBAD
### NOTE: WAS ABLE TO FIND ALL TARGETS, MULTIPLE MATCHES WERE RECONCILED WITH NAMES
t['pmRA'], t['pmDE'] = get_pm_from_simbad(t['SSTc2d'], t['_RA'], t['_DE'], coord_form = 'deg', arcsec_rad=2.0)

### REPLACE THOSE WITH MISSING PMS TO WITH MEDIAN VALUES
### OK BECAUSE WE ARE JUST USING THESE AS STARTING POINT FOR GAIA MATCHING
ind1 = np.where((t['pmRA'] == -99.0) & (t['pmDE'] == -99.0))
ind2 = np.where((t['pmRA'] != -99.0) & (t['pmDE'] != -99.0))
t['pmRA'][ind1] = np.median(t['pmRA'][ind2])
t['pmDE'][ind1] = np.median(t['pmDE'][ind2])


J162221.0-230402: found multiple matches within 2.0 arcsec, keeping name match
        MAIN_ID         DISTANCE_RESULT    PMRA     PMDEC  
                             arcsec      mas / yr  mas / yr
----------------------- --------------- --------- ---------
2MASS J16222099-2304025          0.3680    -9.690   -26.019

J162225.2-240514: found multiple matches within 2.0 arcsec, keeping name match
  MAIN_ID   DISTANCE_RESULT   PMRA    PMDEC  
                 arcsec     mas / yr mas / yr
----------- --------------- -------- --------
NAME Oph 11          0.1974       --       --

J162459.8-245601: found multiple matches within 2.0 arcsec, keeping name match
MAIN_ID DISTANCE_RESULT   PMRA    PMDEC  
             arcsec     mas / yr mas / yr
------- --------------- -------- --------
 WSB 18          0.8158       --       --

J162502.1-245932: found multiple matches within 2.0 arcsec, keeping name match
    MAIN_ID     DISTANCE_RESULT    PMRA     PMDEC  
                     arcsec      mas

In [30]:
### EDR3 (NOTE: SOME -99.0 FOR NOT FOUND, SOME NAN FROM GAIA TABLE HAVING STAR MATCH BUT NO MEASURED PM/PLX)
t['ID_EDR3'], t['PLX_EDR3'], t['ePLX_EDR3'], t['PMRA_EDR3'], t['PMDE_EDR3'], t['RUWE_EDR3'] = get_edr3(t['SSTc2d'], t['_RA'], t['_DE'], 
                                                                                                       t['pmRA'], t['pmDE'], coord_form='deg')

J162145.1-234232: could not find EDR3 match within 2.0 arcsec
J162305.9-233818: could not find EDR3 match within 2.0 arcsec
J162332.2-242554: could not find EDR3 match within 2.0 arcsec
J162522.6-243453: could not find EDR3 match within 2.0 arcsec
J162546.6-242336: could not find EDR3 match within 2.0 arcsec
J162604.6-241751: could not find EDR3 match within 2.0 arcsec
J162607.0-242724: could not find EDR3 match within 2.0 arcsec
J162610.3-242055: could not find EDR3 match within 2.0 arcsec
J162617.2-242345: could not find EDR3 match within 2.0 arcsec
J162618.1-243033: could not find EDR3 match within 2.0 arcsec
J162618.6-242951: could not find EDR3 match within 2.0 arcsec
J162618.8-242610: could not find EDR3 match within 2.0 arcsec
J162619.0-242414: could not find EDR3 match within 2.0 arcsec
J162620.8-242839: could not find EDR3 match within 2.0 arcsec
J162622.3-242407: could not find EDR3 match within 2.0 arcsec
J162622.9-242846: could not find EDR3 match within 2.0 arcsec
J162623.

In [31]:
### GET 2MASS NAMES
t['_2MASS'] = get_2mass_from_coords(t['SSTc2d'], t['_RA'], t['_DE'], coord_form='deg', arcsec_rad = 2.0)

### CHECK FOR DUPLICATES
### NOTE: THESE MATCH THE SSTc2d DUPLICATES (SEE BELOW)
dup = [item for item, count in collections.Counter(t['_2MASS']).items() if count > 1]
print(len(dup))
dup

13


['J16250208-2459323',
 'J16253812-2422362',
 'J16262335-2420597',
 'J16263713-2415599',
 'J16264285-2420299',
 'J16264848-2428389',
 'J16271545-2426398',
 'J16271757-2428562',
 'J16273982-2443150',
 'J16274028-2422040',
 'J16284527-2428190',
 'J16313087-2424399',
 'J16315211-2456156']

In [32]:
### CHECK FOR DUPLICATES IN SSTc2d
### THESE ARE THE BINARY (11 in Sample A, 1 in Sample B) AND TRIPLE (1 in Sample A) SYSTEMS --- see 'Class' column
dup = [item for item, count in collections.Counter(t['SSTc2d']).items() if count > 1]
print(len(dup))
for i, val in enumerate(dup):
    ind = np.where(t['SSTc2d'] == val)
    print(f"{val}\t{t[ind]['_2MASS'][0]}\t{t[ind]['Sample'][0]}\t{[x.strip().decode() for x in t[ind]['Class'].data]}")

13
J162502.1-245932	J16250208-2459323	A	['b', 'II']
J162538.1-242236	J16253812-2422362	A	['b', 'II']
J162623.4-242101	J16262335-2420597	A	['b', 'II']
J162637.1-241560	J16263713-2415599	B	['b', 'II']
J162642.9-242030	J16264285-2420299	A	['b', 'II']
J162648.5-242839	J16264848-2428389	A	['F', 'b']
J162715.4-242640	J16271545-2426398	A	['F', 'b']
J162717.6-242856	J16271757-2428562	A	['I', 'b', 'c']
J162739.8-244315	J16273982-2443150	A	['F', 'b']
J162740.3-242204	J16274028-2422040	A	['b', 'II']
J162845.3-242819	J16284527-2428190	A	['b', 'II']
J163130.9-242440	J16313087-2424399	A	['b', 'II']
J163152.1-245616	J16315211-2456156	A	['F', 'b']


In [33]:
for i, val in enumerate(dup):
    ind = np.where(t['SSTc2d'] == val)
    print(val, t[ind]['_2MASS'][0], t[ind]['Sample'][0], [x.strip().decode() for x in t[ind]['Class'].data])

J162502.1-245932 J16250208-2459323 A ['b', 'II']
J162538.1-242236 J16253812-2422362 A ['b', 'II']
J162623.4-242101 J16262335-2420597 A ['b', 'II']
J162637.1-241560 J16263713-2415599 B ['b', 'II']
J162642.9-242030 J16264285-2420299 A ['b', 'II']
J162648.5-242839 J16264848-2428389 A ['F', 'b']
J162715.4-242640 J16271545-2426398 A ['F', 'b']
J162717.6-242856 J16271757-2428562 A ['I', 'b', 'c']
J162739.8-244315 J16273982-2443150 A ['F', 'b']
J162740.3-242204 J16274028-2422040 A ['b', 'II']
J162845.3-242819 J16284527-2428190 A ['b', 'II']
J163130.9-242440 J16313087-2424399 A ['b', 'II']
J163152.1-245616 J16315211-2456156 A ['F', 'b']


In [53]:
### PRINT FOR TABLE
### NEED TO ADD FINAL DISTANCE AND DUST MASS

print(f"Region\tSource\t2MASS/SSTc2d\tRA\tDec\tEDR3_plx\tEDR3_err_plx\tEDR3_dist_pc\tEDR3_RUWE\tEDR3_Dist_FINAL\tEDR3_Dist_PPVII\tPM_EDR3_mas \
        \tDisk\tDisk_Flag\tObservatory\tFreq_GHz\tWavelength\tBeam_arcsec\tRMS_mJy/beam\tSurvey_Ref\tFlux_mJy\teFlux_mJy\tFlux_mJy_Standardized")
      
for i, val in enumerate(t['SSTc2d']):

    # ADD COMPANION NUMBERS TO NAMES
    if ((t['Class'][i].strip() == 'b') | (t['Class'][i].strip() == 'c')):
        targ = val + ' ' + t['Class'][i].strip()
    else:
        targ = val
    
    ### FORMATTING FOR PRINTING
    tmass = f"{t['_2MASS'][i]}"
    radec = f"{t['RA_J2000'][i]}\t{t['DE_J2000'][i]}"
    obs = f"[, ALMA]"
    survey = f"[, {t['Survey_B6'][i]}]"
    beam = f"[, {t['Beam_B6'][i]}]"
    frq = f"[, {t['Freq_B6'][i]}]"
    wav = f"[, {t['Wave_B6'][i]}]" 
    rms = f"[, {t['RMS_B6'][i]:0.2f}]".replace('-99.00', '??')
    flx = f"[, {t['F225GHz'][i]:0.2f}]"
    eflx = f"[, {t['e_F225GHz'][i]:0.2f}]"
    pm = f"[{t['PMRA_EDR3'][i]}, {t['PMDE_EDR3'][i]}]".replace('-99.0', '').replace('[, ]', '--') 
    plx = f"{t['PLX_EDR3'][i]:0.4f}".replace('-99.0000', '--')
    eplx = f"{t['ePLX_EDR3'][i]:0.4f}".replace('-99.0000', '--')
    ruwe = f"{t['RUWE_EDR3'][i]:0.3f}".replace('-99.000', '--')
    dist = f"{1e3/t['PLX_EDR3'][i]:0.2f}".replace('-10.10', '--')

    ### STANDARDIZE FLUXES
    if t['F225GHz'][i] / t['e_F225GHz'][i] > 3.0:
        sflux = f"[, {t['F225GHz'][i]:0.2f}]"
    else:
        sflux = f"[, <{3.0 * t['e_F225GHz'][i]:0.2f}]"
      
    # DISK TYPE & FLAG
    val1 = (t['Class'][i]).strip()
    if ((val1 == 'II') | (val1 == 'F')):
        dtype, dflag = val1, "1"
    elif ((val1 == 'I') | (val1 == 'III')):
        dtype, dflag = val1, "0"
    elif ((val1 == 'b') | (val1 == 'c')):
        dtype, dflag = val1, "0"
    else:
        dtype, dflag = val1, "???"
        
        
    ### PRINT OUT
    print(f"rOph\t{targ}\t{tmass}\t{radec}\t{plx}\t{eplx}\t{dist}\t{ruwe} \
            \t{'--'}\t{'--'}\t{pm}\t{dtype}\t{dflag}\t{obs}\t{frq}\t{wav}\t{beam}\t{rms} \
            \t{survey}\t{flx}\t{eflx}\t{sflux}")    
    
    # ### FINAL DISTANCE
    # mdist = np.nanmedian(1e3/t['PLX_EDR3'][t['PLX_EDR3'] != -99.0])
    # dist = 1e3/t['PLX_EDR3'][i]
    # if (t['PLX_EDR3'][i] == -99.0):
    #     dist = mdist
    # if ((dist < mdist + 50) & (dist > mdist-50)):
    #     print(f"{dist:0.2f}")
    # else:
    #     print('--')


Region	Source	2MASS/SSTc2d	RA	Dec	EDR3_plx	EDR3_err_plx	EDR3_dist_pc	EDR3_RUWE	EDR3_Dist_FINAL	EDR3_Dist_PPVII	PM_EDR3_mas         	Disk	Disk_Flag	Observatory	Freq_GHz	Wavelength	Beam_arcsec	RMS_mJy/beam	Survey_Ref	Flux_mJy	eFlux_mJy	Flux_mJy_Standardized
rOph	J162034.2-242613	J16203419-2426134	16:20:34.200	-24:26:13.000	--	--	--	--             	--	--	--	II	1	[, ALMA]	[, 225.4]	[, 1.33]	[, 0.98x0.74]	[, ??]             	[, C-W+2019]	[, 0.22]	[, 0.11]	[, <0.33]
rOph	J162118.5-225458	J16211848-2254578	16:21:18.500	-22:54:58.000	7.1463	0.0394	139.93	0.991             	--	--	[-8.584, -24.36]	II	1	[, ALMA]	[, 225.4]	[, 1.33]	[, 0.98x0.74]	[, ??]             	[, C-W+2019]	[, 3.26]	[, 0.13]	[, 3.26]
rOph	J162131.9-230140	J16213192-2301403	16:21:31.900	-23:01:40.000	7.3550	0.0827	135.96	1.292             	--	--	[-8.269, -25.186]	II	1	[, ALMA]	[, 225.4]	[, 1.33]	[, 0.28x0.19]	[, 0.15]             	[, C-W+2019]	[, 4.60]	[, 0.16]	[, 4.60]
rOph	J162138.7-225328	J16213872-2253285	16:21:38.700	-22:5

-99.0

# Lupus

In [19]:
### BAND 7 SURVEY
t7 = Table.read('./Data/2016_Ansdell_Lupus_Table2.fit')
t7 = t7['Name', 'F890', 'e_F890', 'rms']
t7['Name'] = [x.replace(' ', '') for x in t7['Name']]
t7['rms'].name = 'rms_F890'

### BAND 6 SURVEY
t6 = Table.read('./Data/2018_Ansdell_Lupus_Table1.fit')
t6 = t6['Name', 'Fcont', 'e_Fcont', 'rms', 'RAJ2000', 'DEJ2000']
t6['Name'] = [x.replace(' ', '') for x in t6['Name']]
t6['rms'].name = 'rms_F1300'
t6['Fcont'].name = 'F1300'
t6['e_Fcont'].name = 'e_F1300'

### JOIN SURVEYS
t = join(t6, t7, keys='Name', join_type='outer')
t.sort('RAJ2000')

### REMOVE THOSE FOUND NOT TO BE IN LUPUS BETWEEN SURVEYS
t = t[t['Name'] != 'J16104536-3854547']
t = t[t['Name'] != 'J16121120-3832197']

### ADD REFERENCES
t['Survey_B7'] = 'Ansdell+2016'
t['Survey_B6'] = 'Ansdell+2018'
t['Survey_B7'] = t['Survey_B7'].astype('object')
t['Survey_B6'] = t['Survey_B6'].astype('object')

### ADD FREQUENCY AND WAVELENGTH (FROM PAPERS)
t['Freq_B7'] = '335.8'
t['Freq_B6'] = '225.66'
t['Wave_B7'] = '0.890'
t['Wave_B6'] = '1.33'

### MAKE SURE FINAL TABLE IS SAME AS BAND 6 SURVEY (COMPLETE)
print(len(t7), len(t6), len(t))

89 95 95


In [20]:
### CHANGE INFO FOR SOURCES TAKEN FROM OTHER SURVEYS

### REFERENCES
t['Survey_B7'][t['Name'] == 'Sz91'] = 'Tsukagoshi+2019'
t['Survey_B7'][t['Name'] == 'Sz82'] = 'Cleeves+2016'
t['Survey_B6'][t['Name'] == 'Sz82'] = 'Cleeves+2016'

### FREQUENCIES AND WAVELENGTHS
t['Freq_B7'][t['Name'] == 'Sz91'] = '350' ### PAPER DIDN'T SPECIFY BANDWIDTH WEIGHTED, NO DECIMAL POINTS EITHER
t['Wave_B7'][t['Name']   == 'Sz91'] = '??' ### PAPER ONLY USED FREQUENCY
t['Freq_B7'][t['Name'] == 'Sz82'] = '??' ### ONLY GAVE ROUGH WAVELENGTH IN PAPER
t['Freq_B6'][t['Name'] == 'Sz82'] = '??' ### ONLY GAVE ROUGH WAVELENGTH IN PAPER
t['Wave_B7'][t['Name']   == 'Sz82'] = '0.875' ### ONLY GAVE ROUGH WAVELENGTH IN PAPER
t['Wave_B6'][t['Name']    == 'Sz82'] = '1.3' ### ONLY GAVE ROUGH WAVELENGTH IN PAPER

### FLUXES
t['rms_F890'][t['Name'] == 'Sz82'] = 0.21
t['rms_F1300'][t['Name'] == 'Sz82'] = 0.14
t['F890'][t['Name'] == 'Sz82'] = 590
t['F1300'][t['Name'] == 'Sz82'] = 200
t['e_F890'][t['Name'] == 'Sz82'] = 90
t['e_F1300'][t['Name'] == 'Sz82'] = 30
t['rms_F890'][t['Name'] == 'Sz91'] = 0.06
t['F890'][t['Name'] == 'Sz91'] = 45.2
t['e_F890'][t['Name'] == 'Sz91'] = 0.5

In [21]:
### NEEDED TO CHANGE SOME FORMATS TO COMBINE WITH OTHER DATA (SEE BELOW)
t['F890'] = [f"{x:0.2f}" for x in (t['F890'])]
t['e_F890'] = [f"{x:0.2f}" for x in (t['e_F890'])]
t['RAJ2000'] = [str(x.replace(' ',':')) for x in t['RAJ2000']]
t['DEJ2000'] = [str(x.replace(' ',':')) for x in t['DEJ2000']]
t['Name'] = t['Name'].astype('object')

/var/folders/7z/sdhmgb953kj87rmxl40lpvvw0000gp/T/ipykernel_20230/603803750.py:2: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  t['F890'] = [f"{x:0.2f}" for x in (t['F890'])]
/var/folders/7z/sdhmgb953kj87rmxl40lpvvw0000gp/T/ipykernel_20230/603803750.py:3: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  t['e_F890'] = [f"{x:0.2f}" for x in (t['e_F890'])]


In [25]:
### GET NEW DATA (BD + COMPLETION SURVEY)
t_s19a = Table.read('./Data/2019_Sanchis_Lupus_BD_Table1.txt', format='ascii.basic', delimiter='\t')
t_s19b = Table.read('./Data/2019_Sanchis_Lupus_BD_Table2.txt', format='ascii.basic', delimiter='\t')
t_s19 = join(t_s19a, t_s19b)
t_s19.sort('RA_J2000')

### FIX NAMES
t_s19['Object'] = t_s19['Object'].astype('object')
t_s19['Object'][t_s19['Object'] == 'GQ Lup'] = 'Sz75'
t_s19['Object'][t_s19['Object'] == 'EX Lup'] = 'EXLup'
t_s19['Object'][t_s19['Object'] == 'RXJ1556.1-3655'] = 'J15560210-3655282'
t_s19['Object'][t_s19['Object'] == 'V1094 Sco'] = 'V1094Sco'

### ADD BROWN DWARF DATA
### NOTE: WEIGHTED BANDWIDTH CALCULATED MYSELF FROM WHAT SPECTRAL SETUP DESCRIPTION IN PAPER
t_bd = t_s19[t_s19['Survey'] == 'Sanchis+19']
for i, val in enumerate(t_bd['Object']):
    t.add_row([val, -99.0, -99.0, -99.0, t_bd['RA_J2000'][i], t_bd['DE_J2000'][i], t_bd['F_cont'][i], '??', t_bd['rms'][i], 
               'Sanchis+2019', '', '??', '', '0.890', ''])

### ADD COMPLETION DATA BAND 7 ONLY
t_cd = t_s19[t_s19['Survey'] == 'vanTerwisga+19']
for i, val in enumerate(t_cd['Object']):
    ind = np.where(t['Name'] == val)
    # t['RAJ2000'][ind] = t_cd['RA_J2000'][i]
    # t['DEJ2000'][ind] = t_cd['DE_J2000'][i]
    t['F890'][ind] = t_cd['F_cont'][i]
    t['e_F890'][ind] = '??'
    t['rms_F890'][ind] = t_cd['rms'][i]
    t['Survey_B7'][ind] = 'Sanchis+2019'
    t['Freq_B7'][ind] = '??'
    t['Wave_B7'][ind] = '0.890'
    
### SHOULD BE 100
print(len(t))

100


In [27]:
### PRINT TO CHECK
t[10:15]

Name,F1300,e_F1300,rms_F1300,RAJ2000,DEJ2000,F890,e_F890,rms_F890,Survey_B7,Survey_B6,Freq_B7,Freq_B6,Wave_B7,Wave_B6
,mJy,mJy,mJy / beam,,,,,mJy / beam,,,,,,
object,float64,float64,float64,str12,str13,str6,str5,float32,object,object,str5,str6,str5,str4
Sz72,5.54,0.11,0.11,15:47:50.608,-35:28:35.779,14.10,0.28,0.23,Ansdell+2016,Ansdell+2018,335.8,225.66,0.890,1.33
Sz73,10.84,0.18,0.11,15:47:56.922,-35:14:35.185,30.43,0.55,0.39,Ansdell+2016,Ansdell+2018,335.8,225.66,0.890,1.33
Sz74,8.2,0.3,0.13,15:48:05.213,-35:15:53.342,20.94,0.27,0.29,Ansdell+2016,Ansdell+2018,335.8,225.66,0.890,1.33
Sz75,34.1,0.2,0.09,15:49:12.086,-35:39:05.463,78.00,??,0.34,Sanchis+2019,Ansdell+2018,??,225.66,0.890,1.33
Sz76,4.7,0.3,0.08,15:49:30.719,-35:49:51.825,12.00,??,0.35,Sanchis+2019,Ansdell+2018,??,225.66,0.890,1.33


In [31]:
### GET 2MASS NAMES
t['_2MASS'] = get_2mass_from_coords(t['Name'], t['RAJ2000'], t['DEJ2000'], coord_form='hmsdms', arcsec_rad = 2.0)

### FIX THOSE THAT COULD NOT BE FOUND
t['_2MASS'][t['Name'] == 'J160828.1-391310'] = 'J160828.1-391310' ### binary companion with SSTc2d number
t['_2MASS'][t['Name'] == 'J160831.1-385600'] = 'J160831.1-385600' ### only SSTc2d number
t['_2MASS'][t['Name'] == 'Sz108B'] = 'J160842.9-390615' ### only SSTc2d number
t['_2MASS'][t['Name'] == 'J160934.2-391513'] = 'J160934.2-391513' ### only SSTc2d number
t['_2MASS'][t['Name'] == 'Sz123B'] ='J16105158-3853137' ### binary companion without SSTc2d number USING SAME FOR PRIMARY AND SECONDARY
t['_2MASS'][t['Name'] == 'SONYC-Lup3-10'] = '--' ### NO 2MASS OR SSTc2d
# t['_2MASS'][t['Name'] == 'Sz88B'] = 'J16070061-3902194' ### binary companion without SSTc2d number USING SAME FOR PRIMARY AND SECONDARY

J160828.1-391310: could not find 2MASS match within 2.0 arcsec
J160831.1-385600: could not find 2MASS match within 2.0 arcsec
Sz108B: could not find 2MASS match within 2.0 arcsec
J160934.2-391513: could not find 2MASS match within 2.0 arcsec
Sz123B: could not find 2MASS match within 2.0 arcsec
SONYC-Lup3-10: could not find 2MASS match within 2.0 arcsec


In [32]:
### GET PROPER MOTIONS FROM SIMBAD
### NOTE: WAS ABLE TO FIND ALL TARGETS, MULTIPLE MATCHES WERE RECONCILED WITH NAMES
t['pmRA'], t['pmDE'] = get_pm_from_simbad(t['_2MASS'], t['RAJ2000'], t['DEJ2000'], coord_form = 'hmsdms')

### REPLACE THOSE WITH MISSING PMs WITH MEDIAN VALUES
### OK BECAUSE WE ARE JUST USING THESE AS STARTING POINT FOR GAIA MATCHING
ind1 = np.where((t['pmRA'] == -99.0) & (t['pmDE'] == -99.0))
ind2 = np.where((t['pmRA'] != -99.0) & (t['pmDE'] != -99.0))
t['pmRA'][ind1] = np.median(t['pmRA'][ind2])
t['pmDE'][ind1] = np.median(t['pmDE'][ind2])


J15430131-3409153: found multiple matches within 2.0 arcsec, keeping name match
        MAIN_ID         DISTANCE_RESULT   PMRA    PMDEC  
                             arcsec     mas / yr mas / yr
----------------------- --------------- -------- --------
2MASS J15430131-3409153          0.3416       --       --

J15475693-3514346: found multiple matches within 2.0 arcsec, keeping name match
MAIN_ID  DISTANCE_RESULT    PMRA     PMDEC  
              arcsec      mas / yr  mas / yr
-------- --------------- --------- ---------
THA 15-5          0.4672   -14.203   -22.264

J15555030-3801329: found multiple matches within 2.0 arcsec, keeping name match
 MAIN_ID  DISTANCE_RESULT    PMRA     PMDEC  
               arcsec      mas / yr  mas / yr
--------- --------------- --------- ---------
THA 15-10          0.6537   -18.700   -32.700

J15560921-3756057: found multiple matches within 2.0 arcsec, keeping name match
 MAIN_ID  DISTANCE_RESULT    PMRA     PMDEC  
               arcsec      mas / y

In [34]:
### GET EDR3 DATA
t['ID_EDR3'], t['PLX_EDR3'], t['ePLX_EDR3'], t['PMRA_EDR3'], t['PMDE_EDR3'], t['RUWE_EDR3'] = get_edr3(t['_2MASS'], t['RAJ2000'], t['DEJ2000'], 
                                                                                                       t['pmRA'], t['pmDE'], coord_form='hmsdms', arcsec_rad=2.0)

J15430131-3409153: could not find EDR3 match within 2.0 arcsec
J15450634-3417378: could not find EDR3 match within 2.0 arcsec
J15491210-3539051: could not find EDR3 match within 2.0 arcsec
J16011549-4152351: could not find EDR3 match within 2.0 arcsec
J16075475-3915446: could not find EDR3 match within 2.0 arcsec
J16080175-3912316: could not find EDR3 match within 2.0 arcsec
J160831.1-385600: could not find EDR3 match within 2.0 arcsec
J16085828-3907355: could not find EDR3 match within 2.0 arcsec
J16085834-3907491: could not find EDR3 match within 2.0 arcsec
J16091644-3904438: could not find EDR3 match within 2.0 arcsec
J16092032-3904015: could not find EDR3 match within 2.0 arcsec
J16092317-3904074: could not find EDR3 match within 2.0 arcsec
J160934.2-391513: could not find EDR3 match within 2.0 arcsec
J16093928-3904316: could not find EDR3 match within 2.0 arcsec
--: could not find EDR3 match within 2.0 arcsec


In [134]:
### PRINT FOR TABLE
### NEED TO ADD FINAL DISTANCE AND DUST MASS

print(f"Region\tSource\t2MASS/SSTc2d\tRA\tDec\tEDR3_plx\tEDR3_err_plx\tEDR3_dist_pc\tEDR3_RUWE\tEDR3_Dist_FINAL\tEDR3_Dist_PPVII\tPM_EDR3_mas \
        \tDisk\tDisk_Flag\tObservatory\tFreq_GHz\tWavelength\tBeam_arcsec\tRMS_mJy/beam\tSurvey_Ref\tFlux_mJy\teFlux_mJy\tFlux_mJy_Standardized")
      
for i, val in enumerate(t['Name']):

    ### FORMATTING FOR PRINTING
    survey = f"[{t['Survey_B7'][i]}, {t['Survey_B6'][i]}]"
    radec = f"{t['RAJ2000'][i]}\t{t['DEJ2000'][i]}"
    tmass = f"{t['_2MASS'][i]}"
    obs = f"[ALMA, ALMA]"
    frq = f"[{t['Freq_B7'][i]}, {t['Freq_B6'][i]}]"
    wav = f"[{t['Wave_B7'][i]}, {t['Wave_B6'][i]}]"
    rms = f"[{t['rms_F890'][i]:0.2f}, {t['rms_F1300'][i]:0.2f}]".replace('-99.00', '')
    plx = f"{t['PLX_EDR3'][i]:0.4f}".replace('-99.0000', '--')
    eplx = f"{t['ePLX_EDR3'][i]:0.4f}".replace('-99.0000', '--')
    ruwe = f"{t['RUWE_EDR3'][i]:0.3f}".replace('-99.000', '--')
    dist = f"{1e3/t['PLX_EDR3'][i]:0.2f}".replace('-10.10', '--')
    pm = f"[{t['PMRA_EDR3'][i]}, {t['PMDE_EDR3'][i]}]".replace('-99.0', '')
    flx = f"[{t['F890'][i]}, {t['F1300'][i]:0.2f}]".replace('-99.00', '')
    eflx = f"[{t['e_F890'][i]}, {t['e_F1300'][i]:0.2f}]".replace('-99.00', '')
    
    # if val == 'Sz82':
    #     flx = f"[{t['F890'][i]:0.0f}, {t['F1300'][i]:0.0f}]"
    
    ### STANDARDIZE FLUXES
    if t['e_F890'][i] != '??':
        if float(t['F890'][i]) / float(t['e_F890'][i]) > 3.0:                    
            F7 = f"{t['F890'][i]}"
        else:
            F7 = f"<{3.0 * float(t['e_F890'][i]):0.2f}"
    else:
            F7 = f"{t['F890'][i].replace('<=', '<')}"
    if t['e_F1300'][i] == -99.0:
        F6 = ''
    elif t['F1300'][i] / t['e_F1300'][i] > 3.0:
        F6 = f"{t['F1300'][i]:0.2f}"
    else:
        F6 = f"<{3.0 * t['e_F1300'][i]:0.2f}"
    sflux = f"[{F7}, {F6}]"
    

    ### PRINT OUT
    print(f"Lupus\t{val}\t{tmass}\t{radec}\t{plx}\t{eplx}\t{dist}\t{ruwe} \
            \t{'--'}\t{'--'}\t{pm}\t{'DIKSHOLD'}\t{'FLAG'}\t{obs}\t{frq}\t{wav}\t{'BEAM'}\t{rms} \
            \t{survey}\t{flx}\t{eflx}\t{sflux}")
    

Region	Source	2MASS/SSTc2d	RA	Dec	EDR3_plx	EDR3_err_plx	EDR3_dist_pc	EDR3_RUWE	EDR3_Dist_FINAL	EDR3_Dist_PPVII	PM_EDR3_mas         	Disk	Disk_Flag	Observatory	Freq_GHz	Wavelength	Beam_arcsec	RMS_mJy/beam	Survey_Ref	Flux_mJy	eFlux_mJy	Flux_mJy_Standardized
Lupus	Sz65	J15392776-3446171	15:39:27.753	-34:46:17.577	6.5159	0.0253	153.47	1.417             	--	--	[-13.218, -22.121]	DIKSHOLD	FLAG	[ALMA, ALMA]	[335.8, 225.66]	[0.890, 1.33]	BEAM	[0.30, 0.12]             	[Ansdell+2016, Ansdell+2018]	[64.49, 29.94]	[0.32, 0.20]	[64.49, 29.94]
Lupus	Sz66	J15392828-3446180	15:39:28.264	-34:46:18.450	6.4134	0.0269	155.92	1.200             	--	--	[-13.124, -21.807]	DIKSHOLD	FLAG	[ALMA, ALMA]	[335.8, 225.66]	[0.890, 1.33]	BEAM	[0.27, 0.13]             	[Ansdell+2016, Ansdell+2018]	[14.78, 6.42]	[0.29, 0.18]	[14.78, 6.42]
Lupus	J15430131-3409153	J15430131-3409153	15:43:01.290	-34:09:15.400	--	--	--	--             	--	--	[, ]	DIKSHOLD	FLAG	[ALMA, ALMA]	[335.8, 225.66]	[0.890, 1.33]	BEAM	[0.39, 0.12]     